
## Download user ratings for selected boardgames from boardgamegeek.com

### Springboard Capstone 2 project: building a recommendation engine
### John Burt

#### Procedure:

- Load previously downloaded game ID list. All games with > 100 ratings were collected.
- Use [BGG API 2 interface](https://boardgamegeek.com/wiki/page/BGG_XML_API2) to collect user rating data for each game.
- Save ratings to a CSV file.

Notes:

- [BGG API package (not used here) is an alternative.](https://boardgamegeek.com/wiki/page/BGG_XML_API2)
    - installation: pip install boardgamegeek2


- Code used in this notebook is modified from [Building a boardgamegeek.com Data Set with Scraping and APIs in Python](https://sdsawtelle.github.io/blog/output/boardgamegeek-data-scraping.html)



In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pickle
from time import sleep
import timeit

%matplotlib inline

datadir = './data/'

gamelistpath = datadir+'bgg_gamelist.csv'
outputpath = datadir+'bgg_user_ratings_v2.csv'


### Load previously downloaded game ID data

In [3]:
games = pd.read_csv(gamelistpath)
print(games.shape)
games.head()


(12600, 6)


,id,name,nrate,pic_url,nrating_pages,nfullpages
0,13,Catan,87850,https://cf.geekdo-images.com/micro/img/e0y6Bog...,878,878
1,822,Carcassonne,87558,https://cf.geekdo-images.com/micro/img/z0tTaij...,875,875
2,30549,Pandemic,86396,https://cf.geekdo-images.com/micro/img/0m3-oqB...,863,863
3,68448,7 Wonders,71600,https://cf.geekdo-images.com/micro/img/h-Ejv31...,716,716
4,36218,Dominion,69929,https://cf.geekdo-images.com/micro/img/VYp2s2f...,699,699


### "safe" request function

Sometimes a server will spazz out and flub your request or sometimes your connection will blink out in the middle of reading a response, so it's good to wrap requests.get() in something a little more fault tolerant:

In [4]:
def request(msg, slp=1):
    '''A wrapper to make robust https requests.'''
    status_code = 500  # Want to get a status-code of 200
    while status_code != 200:
        sleep(slp)  # Don't ping the server too often
        try:
            r = requests.get(msg)
            status_code = r.status_code
            if status_code != 200:
                print('(err=%d)'%(r.status_code),end='')
#                 print("Server Error! Response Code %i. Retrying..." % (r.status_code))
        except:
            print("An exception has occurred, probably a momentory loss of connection. Waiting one seconds...")
            sleep(1)
    return r

### Collect user ratings for all game IDs in the dataset.

Now that we have the integer game IDs, the specific ratings information for any game can be had, in XML form, directly from the API. This XML response is a lot cleaner and easier to parse than the HTML we had to scrape to get our list of game IDs. Each rating lives in its own simple comment type div.

Note:

- This process takes a long time, especially because you have to pause between requests to avoid being punished by the server.
- The script periodically saves the data to a CSV file that keeps a record of how many ratings are left for each game, so that if the process is interrupted, it can be restarted again where it left off.

In [16]:
#############################################################
# Gathering all ratings from all games in game list data set
#############################################################

# copy # pages to working column 
if 'nfullpages' not in games.columns:
    games['nfullpages'] = games['nrating_pages']

write_header = True

chunksize = 250 # number games per chunk

print('reading ratings in chunks of',chunksize,' games:\n\n')

# Get ratings page-by-page for all games, but do it in chunks of games
for nm, grp in games.groupby(np.arange(len(games))//chunksize):
    # hack to pass the first group
    if nm == 0:
        pass
    else:
        # Initialize a DF to hold all the responses for this chunk of games
        df_ratings = pd.DataFrame(columns=["gameid", "username", "rating"], 
                                  index=range(grp["nrate"].sum()+100000))

        # Initialize indices for writing to the ratings dataframe
        dfidx_start = 0
        dfidx = 0

        # For this group of games, make calls until all FULL pages of every game have been pulled
        pagenum = 1
        while len(grp[grp["nfullpages"] > 0]) > 0: 
            # Get a restricted DF with only still-active games (have ratings pages left)
            active_games = grp[grp["nfullpages"] > 0]

            # Set the next chunk of the DF "gameid" column using the list of game IDs
            id_list = []
            for game in active_games["id"]:
                id_list += [game]*100
            dfidx_end = dfidx_start + len(active_games)*100
            df_ratings.iloc[dfidx_start:dfidx_end, df_ratings.columns.get_loc("gameid")] = id_list

            # Make the request with the list of all game IDs that have ratings left
            id_strs = [str(gid) for gid in active_games["id"]]
            gameids = ",".join(id_strs)
            sleep(1.5)  # Keep the server happy
            r = request(
                "http://www.boardgamegeek.com/xmlapi2/thing?id=%s&ratingcomments=1&page=%i" % (
                    gameids, pagenum))
            soup = BeautifulSoup(r.text, "xml")
            comments = soup("comment")

            print('.',end='')

            # Parse the response and assign it into the dataframe
            l1 = [0]*len(active_games)*100
            l2 = [0]*len(active_games)*100
            j = 0
            for comm in comments:
                l1[j] = comm["username"]
                l2[j] = float(comm["rating"])
                j += 1
            df_ratings.iloc[dfidx_start:dfidx_end, df_ratings.columns.get_loc("username")] = l1
            df_ratings.iloc[dfidx_start:dfidx_end, df_ratings.columns.get_loc("rating")] = l2

            grp["nfullpages"] -= 1  # Decrement the number of FULL pages of each game id
            dfidx_start = dfidx_end     
            pagenum += 1  
    #         print("pagenum updated to %i" %(pagenum,))

        # Strip off the empty rows
        df_ratings = df_ratings.dropna(how="all")

        # Write this batch of all FULL pages of ratings for this chunk of games to the DB
        # df_ratings.to_sql(name="data", con=connex, if_exists="append", index=False)    
        if write_header:
            with open(outputpath, 'w') as f:
                df_ratings.to_csv(f, index=False, encoding="latin-1")
            write_header = False
        else:
            with open(outputpath, 'a') as f:
                df_ratings.to_csv(f, header=False, index=False, encoding="latin-1")

        # save the game df so that I can stop and restart from where I left off
        games.to_csv(gamelistpath, index=False, encoding="utf-8")

        print("\nProcessed %d ratings for batch #%i of games.\n" % (df_ratings.shape[0], nm))

    
print('\n\nDone reading blocks of ratings.')
print('Now read final partial page of ratings for each game:\n')

 

.

C:\Users\john\Anaconda3\envs\datasci\lib\site-packages\ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


........................................................An exception has occurred, probably a momentory loss of connection. Waiting one seconds...
.An exception has occurred, probably a momentory loss of connection. Waiting one seconds...
...............................................
Processed 1936000 ratings for batch #1 of games.

...........................................................
Processed 1187300 ratings for batch #2 of games.

.......................................
Processed 844300 ratings for batch #3 of games.

.............................
Processed 643700 ratings for batch #4 of games.

.......................
Processed 513500 ratings for batch #5 of games.

..................
Processed 424500 ratings for batch #6 of games.

................
Processed 364500 ratings for batch #7 of games.

..............
Processed 315600 ratings for batch #8 of games.

............
Processed 272100 ratings for batch #9 of games.

..........
Processed 238800 ratings for batch #10 of

In [9]:
#############################################################
# Request the final partial page of ratings for each game
#############################################################
# Restore the correct number of FULL pages
# games = pd.read_csv(datadir+'bgg_gamelist.csv')
games["nfullpages"] = (games["nrate"]-50).apply(round, ndigits=-2)/100  # Round DOWN to nearest 100, then divide by 100

# Initialize a DF to hold all the responses over all the chunks of games
df_ratings = pd.DataFrame(columns=["gameid", "username", "rating"], 
                          index=range(len(games)*100*2))

# Initialize indices for writing to the ratings dataframe
dfidx_start = 0
dfidx = 0

# Loop through game-by-game and request the final page of ratings for each game
for idx, row in games.iterrows():
    # Get the game ID and the last page number to request
    pagenum = row["nfullpages"] + 1
    gameid = row["id"]

    # Make the request for just the last page of ratings of this game
    sleep(2)  # Keep the server happy
    comments = None
    try:
        r = requests.get(
            "http://www.boardgamegeek.com/xmlapi2/thing?id=%i&ratingcomments=1&page=%i" % (
                gameid, pagenum))
        soup = BeautifulSoup(r.text, "xml")
        comments = soup("comment")
    except:
        print('\nConnection error: idx=%d, game=%d, page=%d - try one more time:'%(
            idx,gameid,pagenum))
        sleep(5)
        
    if comments is None:
        try:
            r = requests.get(
                "http://www.boardgamegeek.com/xmlapi2/thing?id=%i&ratingcomments=1&page=%i" % (
                    gameid, pagenum))
            soup = BeautifulSoup(r.text, "xml")
            comments = soup("comment")
        except:
            print('\nConnection error: skipping this game')
            sleep(5)

    print('.', end='')

    # Set the next chunk of the DF "gameids" column with this gameid
    if comments is not None:
        dfidx_end = dfidx_start + len(comments)
        id_list = [gameid]*len(comments)
        df_ratings.iloc[dfidx_start:dfidx_end, df_ratings.columns.get_loc("gameid")] = id_list

        # Parse the response and assign it into the dataframe
        l1 = [0]*len(comments)
        l2 = [0]*len(comments)
        j = 0
        for comm in comments:
            l1[j] = comm["username"]
            l2[j] = float(comm["rating"])
            j += 1
        df_ratings.iloc[dfidx_start:dfidx_end, df_ratings.columns.get_loc("username")] = l1
        df_ratings.iloc[dfidx_start:dfidx_end, df_ratings.columns.get_loc("rating")] = l2

        dfidx_start = dfidx_end   # Increment the starting index for next round        

    if idx>0 and idx%1000 == 0:
        print("\nFinished 1000 games - idx =", idx)
        
# Strip off the empty rows
df_ratings = df_ratings.dropna(how="all")

print('Done, ratings =', df_ratings.shape)

# Write this final batch of all partial pages of ratings for this chunk of games to the DB
with open(outputpath, 'a') as f:
    df_ratings.to_csv(f, header=False, index=False, encoding="latin-1")
 

.......................................................................................................................................................................................................................................................................................................................
Connection error: idx=311, game=111661, page=89 - try one more time:
................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................
Connection error: idx=855, game=72225, page=35 - try one more time:
......

...........................................................................................................................................................
Finished 1000 games - idx = 6000
.....
Connection error: idx=6006, game=4746, page=4 - try one more time:
.....................................................................................................................................................................................................................................................................................................................................................................................
Connection error: idx=6379, game=256067, page=3 - try one more time:
..............................................................................................
Connection error: idx=6473, game=44614, page=3 - try one more time:

Connection error: skipping this game
..............................................................................................

................................................................................................................................................................................................................................................................................................................................................................................................................................
Connection error: idx=11820, game=230898, page=2 - try one more time:
.....................................................................................................................................................................................
Finished 1000 games - idx = 12000
............................................
Connection error: idx=12045, game=34277, page=2 - try one more time:
....................................................................................
Connection error: idx=12129, game=118953, page=2 - try one more time:
.....................
Connec

### Clean up saved data and re-save clean version

In [10]:
# read the data I just created, remove duplicates and NaNs, save what's left

# for some reason I have to use this encoding
df = pd.read_csv(outputpath, encoding = "ISO-8859-1")
print('read dataframe =', df.shape)

df.dropna(how="all",inplace=True)
print('after dropna =', df.shape)

df.drop_duplicates(inplace=True)
print('after drop_duplicates =', df.shape)

# save cleaned up dataframe to output file
df.to_csv(outputpath, index=False, encoding="ISO-8859-1")


read dataframe = (20788818, 3)
after dropna = (20788818, 3)
after drop_duplicates = (15314277, 3)
